<a href="https://colab.research.google.com/github/RenYuanXue/LearningBERT/blob/main/Fine%20Tuning%20with%20BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Load CoLA Dataset

Get wget package to download the dataset

In [2]:
!pip install wget

  Created wheel for wget: filename=wget-3.2-cp37-none-any.whl size=9681 sha256=282924960009766d009b0c623b834b22b9773b0a4bd804f63174a64435e6c4e0
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


Download the zip file using wget

In [7]:
import wget
import os

print('Downloading dataset...')

# The url for the dataset zip file.
url = 'https://nyu-mll.github.io/CoLA/cola_public_1.1.zip'

# Download the file (if we haven't already)
if not os.path.exists('./cola_public_1.1.zip'):
  wget.download(url, './cola_public_1.1.zip')

Unzip the dataset to file system

In [8]:
# Unzip the dataset (if we haven't done so).
if not os.path.exists('./cola_public/'):
  !unzip cola_public_1.1.zip

Archive:  cola_public_1.1.zip
   creating: cola_public/
  inflating: cola_public/README      
   creating: cola_public/tokenized/
  inflating: cola_public/tokenized/in_domain_dev.tsv  
  inflating: cola_public/tokenized/in_domain_train.tsv  
  inflating: cola_public/tokenized/out_of_domain_dev.tsv  
   creating: cola_public/raw/
  inflating: cola_public/raw/in_domain_dev.tsv  
  inflating: cola_public/raw/in_domain_train.tsv  
  inflating: cola_public/raw/out_of_domain_dev.tsv  


Load the dataset to pandas dataframe

In [13]:
import pandas as pd

# Load data to pandas dataframe.
df = pd.read_csv('./cola_public/raw/in_domain_train.tsv',
          delimiter = '\t', header = None,
          names = ['sentence_source', 'label', 'label_notes', 'sentence'])

# Report number of sentences in the dataframe.
print('Number of training sentences: {0}'.format(df.shape[0]))

# Display first few rows from the data.
df.head()

Number of training sentences: 8551


,sentence_source,label,label_notes,sentence
0,gj04,1,NaN,"Our friends won't buy this analysis, let alone..."
1,gj04,1,NaN,One more pseudo generalization and I'm giving up.
2,gj04,1,NaN,One more pseudo generalization or I'm giving up.
3,gj04,1,NaN,"The more we study verbs, the crazier they get."
4,gj04,1,NaN,Day by day the facts are getting murkier.


If the sentence is grammatically correct, it is labelled as 1, otherwise 0.

Extract the sentences and labels used for BERT.

In [14]:
sentences = df.sentence.values
labels = df.label.values

## Tokenization & input formatting

Get transformers library in case don't have it.

In [16]:
!pip install transformers

     |████████████████████████████████| 1.8MB 12.8MB/s 
     |████████████████████████████████| 3.2MB 59.9MB/s 
     |████████████████████████████████| 890kB 38.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=de2ff30c8ccc1ef0427455e67ebea431eb573cee72e34e17243dec0702477dce
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


First, we need to load the BERT tokenizer.

In [17]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Apply the tokenizer to our input sentences.

In [18]:
# Print original sentence.
print('Original: ', sentences[0])

# Print splitted sentences as tokens.
print('Tokenized: ', tokenizer.tokenize(sentences[0]))

# Print sentences mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences[0])))

Original:  Our friends won't buy this analysis, let alone the next one we propose.
Tokenized:  ['our', 'friends', 'won', "'", 't', 'buy', 'this', 'analysis', ',', 'let', 'alone', 'the', 'next', 'one', 'we', 'propose', '.']
Token IDs:  [2256, 2814, 2180, 1005, 1056, 4965, 2023, 4106, 1010, 2292, 2894, 1996, 2279, 2028, 2057, 16599, 1012]
